In [1]:
import albumentations as A
import cv2
import json
import numpy as np
import os
import sys

module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

from src.data.image_augmentation import get_bbox_from_json
from src.data.image_augmentation import plot_img_with_bbox

In [2]:
transform = A.Compose([
    A.RandomCrop(width=450, height=450),
    A.HorizontalFlip(p=0.5),
    A.VerticalFlip(p=0.5),
    A.RandomBrightnessContrast(p=0.2),
], bbox_params=A.BboxParams(format='coco'))

In [3]:
bboxes = get_bbox_from_json('Rot1.jpg')

In [4]:
img = cv2.imread("../data/raw/nano/Rot/Rot1.jpg")


In [5]:
transformed = transform(image=img, bboxes=bboxes)
transformed_image = transformed['image']
transformed_bboxes = transformed['bboxes']

In [7]:
plot_img_with_bbox(transformed_image, transformed_bboxes)